In [3]:
import pandas as pd
from pathlib import Path
import sys
import numpy as np

PATH = '/Users/danil/Documents/github/jooble'
sys.path.append(str(PATH))

In [16]:
train = pd.read_csv(PATH + '/data/raw/train.tsv', sep ='\t')

In [17]:
train.head()

,id_job,features
0,1864791934054678713,"2,9835,9999,9941,9945,9386,9899,9421,9954,9952..."
1,-7413918695841089440,"2,9082,9999,9700,9669,9981,9729,9822,9667,9526..."
2,-9223271545392256405,"2,9064,9999,9730,9585,9890,9740,9751,9538,9590..."
3,-9223240803898726824,"2,9402,9999,9711,9742,9975,9728,9924,9692,9514..."
4,-9223102057156184105,"2,9655,9996,9751,9722,9925,9736,9987,9733,9475..."


In [18]:
train.columns

Index(['id_job', 'features'], dtype='object')

In [19]:
train['id_job']

0      1864791934054678713
1     -7413918695841089440
2     -9223271545392256405
3     -9223240803898726824
4     -9223102057156184105
              ...         
794   -9168651328982232304
795   -9168503098892640218
796   -9168200676135763139
797   -9168200673481327350
798   -9168049274035838891
Name: id_job, Length: 799, dtype: int64

требования: 

чтобы читался файл на 50 mln строк - dask

чтобы удобно было добавлть новые паттерны учитывая первое значение в фиче

использование другой нормализации кроме z 

1. открываем train.tsc при помощи (pandas -> dask -> multiprocess. compare)

2. разбиваем фичи на колонки

3. треним нормализатор (сначало просто стандартный нормализатор -> с учетом набора -> делаем удобную добавку в классах)

    - сохраняем нормализатор (если нужно)
    

4. открываем test.tsc

5. разбиваем фичи на колонки

6. оцениваем max_feature_2_index

7. оцениваем max_feature_2_abs_mean_diff

8. нормализируем

9. переименовуем feature_2 на feature_2_stand

10. сохраняем test_proc.tsc c 

    - id_job, 

    - feature_2_stand_{i} double, 

    - max_feature_2_index int - индекс самой большой фичи у этой работы, 
    - max_feature_2_abs_mean_diff - абсолютное отклонение признака с индексом max_feature_2_index от его среднего значения 
mean(feature_2_{max_feature_2_index})

Double – это 64-битная точность

1. открываем train.tsc при помощи (dask -> multiprocess. compare)


3. треним нормализатор (с учетом набора -> делаем удобную добавку в классах)

    - сохраняем нормализатор (если нужно)

Double – это 64-битная точность

# open data

In [61]:
train = pd.read_csv(PATH + '/data/raw/train.tsv', sep ='\t')

# prepare features

In [62]:
def preprocess_data(df):
    parsed_features = df['features'].str.split(',', expand = True).astype(np.int16)
    parsed_features.insert(0, 'id_job', df['id_job'])
    return parsed_features

In [63]:
train_feautres = preprocess_data(train)

In [64]:
train_feautres.head()

,id_job,0,1,2,3,4,5,6,7,8,...,247,248,249,250,251,252,253,254,255,256
0,1864791934054678713,2,9835,9999,9941,9945,9386,9899,9421,9954,...,8818,9954,9925,9934,8689,9958,9086,9114,9950,9875
1,-7413918695841089440,2,9082,9999,9700,9669,9981,9729,9822,9667,...,9979,9752,9695,9676,9974,9788,9955,9907,9747,9824
2,-9223271545392256405,2,9064,9999,9730,9585,9890,9740,9751,9538,...,9930,9705,9645,9652,9954,9582,9947,9876,9722,9791
3,-9223240803898726824,2,9402,9999,9711,9742,9975,9728,9924,9692,...,9959,9737,9611,9661,9912,9768,9963,9971,9767,9809
4,-9223102057156184105,2,9655,9996,9751,9722,9925,9736,9987,9733,...,9908,8208,9631,9571,9971,9407,9964,9990,9783,9907


# fit z scaler

в него передаю датафрейм

он смотрит на значение в первой колонке

если == 2 то применяем один стандартизатор

если == 3 то применяю другой стандартизатор

In [ ]:
class scalers:
    
    def __init__(self):
        pass
    
    def fit(self, X):
        self.mean  = 
        self.model.fit(X, y, batch_size = 32, epochs=self.epoch)
        return self.model
        
    def predict(self, X):
        return self.model.predict_classes(X)
    
    def predict_score(self, X):
        return self.model.predict(X)

In [56]:
from sklearn.preprocessing import StandardScaler

In [59]:
scaler = StandardScaler()
scaler.fit(train_feautres.iloc[:, 2:])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [ ]:
def norm_scaler(df):
    

# open test data

In [65]:
test = pd.read_csv(PATH + '/data/raw/test.tsv', sep ='\t')

In [66]:
test_feautres = preprocess_data(test)

# max_feature_2_index

In [105]:
test_feautres['max_feature_2_index'] = test_feautres.iloc[:, 2:].values.argmax(axis = 1)

# max_feature_2_abs_mean_diff

In [123]:
max_values = test_feautres.iloc[:, 2:258].values.max(axis = 1)
mean_values = test_feautres.iloc[:, 2:258].values.mean(axis = 0)
indexes = test_feautres['max_feature_2_index']

In [129]:
test_feautres['max_feature_2_abs_mean_diff'] = np.abs(max_values - mean_values[indexes])

# feature_2_stand transform

In [139]:
test_feautres.iloc[:, 2:258] = scaler.transform(test_feautres.iloc[:, 2:258])

In [161]:
test_feautres = test_feautres.rename(columns = {i:f'feature_2_stand_{i}' for i in range(1, 257)})
test_feautres = test_feautres.drop(columns=[0])

# save

In [165]:
test_feautres.to_csv(PATH + '/data/processed/test_proc.tsv', sep ='\t', index = False)

In [167]:
pd.read_csv(PATH + '/data/processed/test_proc.tsv', sep ='\t')

,id_job,feature_2_stand_1,feature_2_stand_2,feature_2_stand_3,feature_2_stand_4,feature_2_stand_5,feature_2_stand_6,feature_2_stand_7,feature_2_stand_8,feature_2_stand_9,...,feature_2_stand_249,feature_2_stand_250,feature_2_stand_251,feature_2_stand_252,feature_2_stand_253,feature_2_stand_254,feature_2_stand_255,feature_2_stand_256,max_feature_2_index,max_feature_2_abs_mean_diff
0,-9168029089769934451,0.769135,0.234307,0.720399,0.696881,-0.576915,0.803130,0.258285,0.706847,0.808236,...,0.829606,0.772527,0.120503,0.802841,-0.502010,0.184573,0.798240,0.731307,161,154.17
1,-9167993139315005259,-0.496788,0.234307,-0.162567,0.092944,0.584295,-0.398820,0.294103,0.030519,-0.419534,...,-0.189921,0.021303,0.438698,0.201120,0.419363,0.468833,-0.005895,-0.275757,161,154.17
2,-9167993136660569470,-0.502214,0.234307,-0.447395,-0.091390,0.419332,-0.131720,0.272612,-0.225390,-0.333323,...,-0.358414,-0.323721,0.379037,-0.028107,0.436516,0.305383,-0.277769,-0.890851,161,154.17
3,-9167993126042826314,-0.688485,0.232522,-0.874636,-0.564352,0.545480,-0.622516,0.458866,-0.758096,-0.814918,...,-0.895308,-0.703004,0.546657,-1.034160,0.480624,0.497259,-0.890443,-1.017487,161,154.17
4,-9167914043308884846,-1.004966,-0.204754,-1.263901,-1.144035,0.053826,-0.919665,0.532891,-1.724280,-1.575957,...,-1.035243,-1.767441,0.421652,-1.505349,0.284588,0.520947,-1.380583,-0.348121,203,21.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-9163569547596238325,-0.077225,0.234307,-0.263839,0.075966,0.344937,0.095315,0.325145,-0.468241,-0.229275,...,0.141354,-0.113281,0.521088,-0.231865,0.507579,0.433300,-0.059504,-0.143090,161,154.17
96,-9163525656344858443,-0.044673,0.232522,-0.656268,-0.445505,0.661925,-0.542386,0.532891,-1.907072,-1.159764,...,-1.092359,-0.223395,0.475631,-0.578889,0.529633,0.475939,-0.825347,-1.035578,161,154.17
97,-9163472054053566485,-1.316021,0.218243,-2.032936,-1.988091,0.066764,-3.654100,0.380067,-3.207503,-4.403098,...,-4.776363,-2.521111,0.106298,-1.849190,0.152263,0.438038,-1.943478,-0.577274,161,154.17
98,-9163470125725648647,0.850516,0.236091,0.799518,0.755092,0.173505,0.926664,0.513788,0.756462,0.944985,...,0.923848,0.826360,0.512565,0.866515,0.441417,0.653601,0.874825,0.876035,1,172.12
